In [1]:
import torch
from clip_validation_functions import CLIPS_VALIDATIONS  # Import your validation functions
from base_validation_function import construct_tensor_validator  # Import the validator constructor

# Define the column names in the order expected by the tensor
COLUMN_NAMES = [
    "Saddle height",
    "Seat tube length",
    "Seatpost LENGTH",
    "BSD rear",
    "ERD rear",
    "Head tube length textfield",
    "Head tube lower extension2",
    "Head tube upper extension2",
    "CS textfield"
]

# Create the tensor-based validator
tensor_validator = construct_tensor_validator(CLIPS_VALIDATIONS, COLUMN_NAMES)

# Create a sample input tensor with requires_grad=True for gradient tracking
sample_designs = torch.tensor(
    [[500.0, 450.0, 100.0, 600.0, 580.0, 150.0, 20.0, 30.0, -10.0],  # Design 1
     [520.0, 460.0, 105.0, 610.0, 590.0, 160.0, 25.0, 35.0, 5.0],    # Design 2
     [480.0, 440.0, 95.0, 590.0, 570.0, 140.0, 15.0, 25.0, -5.0]],   # Design 3
    dtype=torch.float32, requires_grad=True  # Enable gradient tracking
)

# Run validation
validation_results = tensor_validator(sample_designs)

# Compute a simple sum loss to check gradients
# loss = validation_results.sum()
loss = validation_results[validation_results > 0].sum()  # Sum only positive results
loss.backward()

# Display validation results
print("Validation Results:")
print(validation_results)

# Display gradients
print("\nGradients with respect to input tensor:")
print(sample_designs.grad)


Validation function [Bsd front too small] encountered exception ['BSD front' is not in list]
Validation Results:
tensor([[ 10.,  80.,  20.,   1., 130., 100.,  10.],
        [ 20.,  75.,  20.,   1., 135., 100.,  -5.],
        [  0.,  85.,  20.,   1., 125., 100.,   5.]], grad_fn=<CopySlices>)

Gradients with respect to input tensor:
tensor([[ 0.,  0.,  1.,  1., -1.,  2., -2., -1., -1.],
        [ 0.,  0.,  1.,  1., -1.,  2., -2., -1.,  0.],
        [-1.,  1.,  1.,  1., -1.,  2., -2., -1., -1.]])


In [2]:
import torch
from aero_validation_functions import AERO_VALIDATIONS  # Import your validation functions
from base_validation_function import construct_tensor_validator  # Import the validator constructor

# Define column names in the correct order
COLUMN_NAMES = [
    'hand_x', 'hand_y', 'hip_x', 'hip_y', 'crank_length', 
    'upper_leg', 'lower_leg', 'arm_length', 'torso_length', 
    'neck_and_head_length', 'torso_width'
]

# Create the tensor validator
tensor_validator = construct_tensor_validator(AERO_VALIDATIONS, COLUMN_NAMES)

def gradient_check(validator, num_samples=5, epsilon=1e-4):
    """
    Checks if gradients are properly flowing through the validation functions.
    
    Parameters:
        validator: The constructed tensor validator function.
        num_samples (int): Number of test samples to check.
        epsilon (float): Small perturbation for numerical gradient comparison.
    
    Returns:
        None (Prints gradient status)
    """
    # Create a random input tensor with requires_grad=True
    sample_designs = torch.randn((num_samples, len(COLUMN_NAMES)), dtype=torch.float32, requires_grad=True)

    # Run the validation function
    validation_results = validator(sample_designs)

    # Compute a simple loss function (sum of all validation outputs)
    loss = validation_results.sum()

    # Backpropagate
    loss.backward()

    # Check if gradients exist and are non-zero
    if sample_designs.grad is None:
        print("❌ No gradients were computed!")
    else:
        # Check for any zero gradients
        grad_norm = sample_designs.grad.abs().sum(dim=0)
        if torch.all(grad_norm < epsilon):
            print("⚠️ Gradients are too small, potential issue with differentiability.")
        else:
            print("✅ Gradients successfully computed!")
        
        # Display gradient values
        print("\nGradient values per input column:")
        for col, grad_val in zip(COLUMN_NAMES, sample_designs.grad.mean(dim=0)):
            print(f"{col}: {grad_val.item():.6f}")

# Run the gradient check
gradient_check(tensor_validator)


✅ Gradients successfully computed!

Gradient values per input column:
hand_x: 0.594541
hand_y: 0.261865
hip_x: 0.603010
hip_y: -0.418754
crank_length: 1.000000
upper_leg: -1.000000
lower_leg: -1.000000
arm_length: -1.000000
torso_length: -1.000000
neck_and_head_length: 0.000000
torso_width: 0.000000
